In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [39]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [40]:
%run -i ../../core.py

In [41]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [42]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)



In [43]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.mean(X))
        # return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_60428/1451395984.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.mean(X))


In [44]:
# prover calculates settings, send to verifier
# In linearity, scale doesnt affect lookup size
theory_output = torch.mean(data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1),torch.mean(X))
        # return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [15], "resources", settings_path)

Theory_output:  tensor(49.9700)
==== Generate & Calibrate Setting ====
scale:  [15]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":15,"param_scale":15,"scale_rebase_multiplier":10,"lookup_range":[0,600],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":604,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,30],"model_input_scales":[15],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"Recip":{"scale":32768.0}}],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_60428/1057230449.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1),torch.mean(X))


In [45]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.5255029201507568 seconds
Theory output:  tensor(49.9700)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.866302490234375
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[3148702633260319778, 12567662588507959291, 5496310355884410072, 3006741169691532095], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [17482780814609032706, 7615535357985293566, 4755406845152436647, 3486774616844464877]]], 'proof': '1b11774a10b3e71bc1e36b4a59598066258b49ef32a8a64ea45bbd53d1fe9710196262993e549845664d9d3b76b4f0de7d204357b6b0e93e04ff6bc227a9959d06a2f8bacdd7671f8e53420f8d3f77830ee2ac1ebc12b887f1c36b55e486a44720161344743abe7efb74492000045d82e269dee03bcab6b30f9e1290f523a96b20a74c4256c0c1a72858993ec8d0c8cfacc1bb339b52502b2961ca6b1f50bec720b53c0f957939a52713f8898f28fc790281f36bff9a206819a0b52d5f66c8eb2e7ec9a0adea38f83468e6c68d37648eec29a76160d7f093a3cb7cd75f487f661a7f3058bca8665787b88108c5ac1dcac0f2d7c3c74774168376f63bd81df5dd18ea93cb121119112fd7083b5bc206d4c45ebb6e3b99a5f6bfa9458a9c53367109afbf6d1fbabbf59e3f24170b6cecfd8117c9d8a953ec75d4174a2d3ad3e7ed08dbe31c561893cad5ad34bbd17ee556bf7127377fc64fa23d56bc1c7b7dd8a

In [46]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[3148702633260319778, 12567662588507959291, 5496310355884410072, 3006741169691532095], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [17482780814609032706, 7615535357985293566, 4755406845152436647, 3486774616844464877]]]
proof boolean:  1.0
proof result 1 : 49.866302490234375
verified
